In [1]:
import numpy as np
import pandas as pd
import cufflinks
import json
import os
from copy import deepcopy
from datetime import datetime

import trading_gym
from trading_gym.ray.logger import calculate_tearsheet, CustomLogger, PPOTensorboard
from trading_gym.registry.gaia.v7.env import GAIAPredictorsContinuousV7
from trading_gym.ray.models import MultiLayersPerceptron

import ray
from ray import rllib
from ray import tune
from ray.rllib.models import ModelCatalog
from ray.rllib.models.model import Model
from ray.rllib.models.misc import normc_initializer, get_activation_fn
from ray import cloudpickle
from ray.utils import binary_to_hex, hex_to_binary

import tensorflow as tf
import tensorflow.contrib.slim as slim

ModelCatalog.register_custom_model(MultiLayersPerceptron.__name__, MultiLayersPerceptron)
cufflinks.go_offline()
ray.init(ignore_reinit_error=True)

2019-07-02 16:32:06,139	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-02_16-32-06_139369_63649/logs.
2019-07-02 16:32:06,263	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:15915 to respond...
2019-07-02 16:32:06,386	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:15196 to respond...
2019-07-02 16:32:06,393	INFO services.py:806 -- Starting Redis shard with 10.0 GB max memory.
2019-07-02 16:32:06,429	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-02_16-32-06_139369_63649/logs.
2019-07-02 16:32:06,431	WARNING services.py:1294 -- Warning: Capping object memory store to 20.0GB. To increase this further, specify `object_store_memory` when calling ray.init() or ray start.
2019-07-02 16:32:06,433	INFO services.py:1442 -- Starting the Plasma object store with 20.0 GB memory using /dev/shm.


{'node_ip_address': '10.0.5.4',
 'redis_address': '10.0.5.4:15915',
 'object_store_address': '/tmp/ray/session_2019-07-02_16-32-06_139369_63649/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-07-02_16-32-06_139369_63649/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2019-07-02_16-32-06_139369_63649'}

In [2]:
from ray.rllib.models import ModelCatalog
from ray.rllib.models.model import Model
from ray.rllib.models.misc import normc_initializer, get_activation_fn
import tensorflow as tf
import tensorflow.contrib.slim as slim


class MLP(Model):
    def _build_layers_v2(self, input_dict: dict, num_outputs: int, config: dict):
        import tensorflow.contrib.slim as slim

        with tf.name_scope("fc_net"):
            last_layer = input_dict['obs']
            activation = get_activation_fn(config.get("fcnet_activation"))
            for i, size in enumerate(config.get("fcnet_hiddens"), 1):
                last_layer = slim.fully_connected(
                    inputs=last_layer,
                    num_outputs=size,
                    weights_initializer=normc_initializer(1.0),
                    activation_fn=activation,
                    scope="fc{}".format(i),
                )
#                 We don't need any dropout at this stage
#                 last_layer = tf.layers.dropout(
#                     inputs=last_layer,
#                     rate=config['custom_options']["fcnet_dropout_rate"],
#                     training=input_dict['is_training'],
#                     name="dropout{}".format(i),
#                 )
            output = slim.fully_connected(
                inputs=last_layer,
                num_outputs=num_outputs,
                weights_initializer=normc_initializer(0.01),
                activation_fn=None,
                scope="fc_out",
            )
            return output, last_layer

ModelCatalog.register_custom_model(MLP.__name__, MLP)

In [3]:
def cloudpickleloads(obj):
    if isinstance(obj, dict):
        try:
            return cloudpickle.loads(hex_to_binary(obj["value"]))
        except:
            for key, value in obj.items():
                if isinstance(value, dict):
                    if sorted(value) == ['_type', 'value']:
                        obj[key] = cloudpickle.loads(hex_to_binary(value["value"]))
                    else:
                        obj[key] = cloudpickleloads(value)
                elif isinstance(value, list):
                    for i, item in enumerate(value):
                        obj[key][i] = cloudpickleloads(item)
    return obj

In [10]:
# This is for the 0.8 clip (the best run)

paths = {2007: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/logs/PPOclip_0.8-WalkForward-750k2007/experiment_state-2019-06-24_23-35-32.json',
        2008: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/logs/PPOclip_0.8-WalkForward-750k2008/experiment_state-2019-06-25_00-28-20.json',
        2009: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/logs/PPOclip_0.8-WalkForward-750k2009/experiment_state-2019-06-25_01-21-47.json',
        2010: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/logs/PPOclip_0.8-WalkForward-750k2010/experiment_state-2019-06-25_02-14-52.json',
        2011: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/logs/PPOclip_0.8-WalkForward-750k2011/experiment_state-2019-06-25_03-08-23.json',
        2012: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/logs/PPOclip_0.8-WalkForward-750k2012/experiment_state-2019-06-25_04-00-54.json',
        2013: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/logs/PPOclip_0.8-WalkForward-750k2013/experiment_state-2019-06-25_04-54-29.json',
        2014: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/logs/PPOclip_0.8-WalkForward-750k2014/experiment_state-2019-06-25_05-47-18.json',
        2015: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/logs/PPOclip_0.8-WalkForward-750k2015/experiment_state-2019-06-25_06-40-12.json',
        2016: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/logs/PPOclip_0.8-WalkForward-750k2016/experiment_state-2019-06-25_07-33-17.json',
        2017: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/logs/PPOclip_0.8-WalkForward-750k2017/experiment_state-2019-06-25_08-26-33.json'
        }

# This is 1.0 clip 
# paths = {2007: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/PPO Tuning/logs/PPOclip_0.8-WalkForward-750k2007/experiment_state-2019-07-01_10-37-58.json',
#         2008: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/PPO Tuning/logs/PPOclip_0.8-WalkForward-750k2008/experiment_state-2019-07-01_12-18-57.json',
#         2009: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/PPO Tuning/logs/PPOclip_0.8-WalkForward-750k2009/experiment_state-2019-07-01_13-41-34.json',
#         2010: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/PPO Tuning/logs/PPOclip_0.8-WalkForward-750k2010/experiment_state-2019-07-01_14-50-33.json',
#         2011: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/PPO Tuning/logs/PPOclip_0.8-WalkForward-750k2011/experiment_state-2019-07-01_15-59-02.json',
#         2012: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/PPO Tuning/logs/PPOclip_0.8-WalkForward-750k2012/experiment_state-2019-07-01_17-19-21.json',
#         2013: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/PPO Tuning/logs/PPOclip_0.8-WalkForward-750k2013/experiment_state-2019-07-02_11-49-44.json',
#         2014: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/PPO Tuning/logs/PPOclip_0.8-WalkForward-750k2014/experiment_state-2019-07-02_12-12-03.json',
#         2015: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/PPO Tuning/logs/PPOclip_0.8-WalkForward-750k2015/experiment_state-2019-07-02_12-42-28.json',
#         2016: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/PPO Tuning/logs/PPOclip_0.8-WalkForward-750k2016/experiment_state-2019-07-02_13-14-05.json',
#         2017: '/home/Nicholas/Desktop/trading-gym/notebooks/registry/gaia/v7/PPO Tuning/logs/PPOclip_0.8-WalkForward-750k2017/experiment_state-2019-07-02_13-46-32.json'
#         }

In [11]:
for year,path in paths.items():
    with open(path) as f:
        metadata = json.load(f)

    runner_data = metadata['runner_data']
    stats = metadata['stats']

    checkpoint = metadata['checkpoints'][-1]
    checkpoint = cloudpickleloads(checkpoint)
    checkpoint_path = cloudpickle.loads(hex_to_binary(checkpoint['_checkpoint'])).value

    config = checkpoint['config']
    print(config)
    break

{'monitor': False, 'log_level': 'INFO', 'callbacks': {'on_episode_start': None, 'on_episode_step': None, 'on_episode_end': None, 'on_sample_end': None, 'on_train_result': tune.function(<function calculate_tearsheet at 0x7feb3f662158>), 'on_postprocess_traj': None}, 'ignore_worker_failures': False, 'model': {'conv_filters': None, 'conv_activation': 'relu', 'fcnet_activation': 'tanh', 'fcnet_hiddens': [256, 256], 'free_log_std': False, 'squash_to_range': False, 'use_lstm': False, 'max_seq_len': 20, 'lstm_cell_size': 256, 'lstm_use_prev_action_reward': False, 'framestack': True, 'dim': 84, 'grayscale': False, 'zero_mean': True, 'custom_preprocessor': None, 'custom_model': 'MLP', 'custom_options': {}}, 'optimizer': {}, 'gamma': 0, 'horizon': None, 'soft_horizon': False, 'env_config': {'folds': {'training-set': [datetime.datetime(1, 1, 1, 0, 0), datetime.datetime(2007, 12, 31, 0, 0)], 'test-set': [datetime.datetime(2008, 1, 1, 0, 0), datetime.datetime(2008, 12, 31, 0, 0)]}}, 'env': <class '

In [12]:
policies = dict()
env_configs = dict()
for year, path in paths.items():
    print(year)
    with open(path) as f:
        metadata = json.load(f)
        checkpoints = metadata['checkpoints']
        runner_data = metadata['runner_data']
        stats = metadata['stats']

    for checkpoint in checkpoints:
        checkpoint = cloudpickleloads(checkpoint)
        cp = cloudpickle.loads(hex_to_binary(checkpoint['_checkpoint']))
        config = checkpoint['config']
        env_cls = config['env']
        env_config = config['env_config']
        path_restore = os.path.join(checkpoint['logdir'], cp.value)
        

    agent = rllib.agents.ppo.PPOTrainer(config, env_cls)
    agent.restore(path_restore)
    
    policies[year] = agent.get_policy()
    env_configs[year] = env_config

2019-07-05 10:09:41,064	WARNING ppo.py:153 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).


2007


2019-07-05 10:09:42,620	INFO policy_evaluator.py:311 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
2019-07-05 10:09:44,485	INFO policy_evaluator.py:731 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.PPOTFPolicy object at 0x7feaf23739b0>}
2019-07-05 10:09:44,488	INFO policy_evaluator.py:732 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7feaf2373588>}
2019-07-05 10:09:44,491	INFO policy_evaluator.py:343 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7fe592248438>}
2019-07-05 10:09:44,650	INFO multi_gpu_optimizer.py:80 -- LocalMultiGPUOptimizer devices ['/cpu:0']


(pid=70711) 2019-07-05 10:09:52,642	INFO policy_evaluator.py:311 -- Creating policy evaluation worker 1 on CPU (please ignore any CUDA init errors)
(pid=70711) 2019-07-05 10:09:52.674854: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=70709) 2019-07-05 10:09:53,085	INFO policy_evaluator.py:311 -- Creating policy evaluation worker 2 on CPU (please ignore any CUDA init errors)
(pid=70709) 2019-07-05 10:09:53.116984: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA


FileNotFoundError: [Errno 2] No such file or directory: 'logs/PPOclip_0.8-WalkForward-750k2007/PPOTrainer_GAIAPredictorsContinuousV7_0_clip_param=0.8,entropy_coeff=1e-05,kl_coeff=0.2,kl_target=0.01,lr=1e-05,num_sgd_iter=8,tra_2019-06-24_23-35-32hw4gynt_/logs/PPOclip_0.8-WalkForward-750k2007/PPOTrainer_GAIAPredictorsContinuousV7_0_clip_param=0.8,entropy_coeff=1e-05,kl_coeff=0.2,kl_target=0.01,lr=1e-05,num_sgd_iter=8,tra_2019-06-24_23-35-32hw4gynt_/checkpoint_188/checkpoint-188.tune_metadata'

In [ ]:
levels = list()
mappings = pd.DataFrame()
mapping_functions = dict()
for year, policy in policies.items():
    print('_______________________________________{}____________________________________________'.format(year))
    
    env_config = env_configs[year]
    env = env_cls(env_config)
    episode = env.sample_episode(
        fold='test-set',
        policy=policy,
    )
    renderer = env.render(
        benchmark=env._load_benchmark().squeeze().loc[str(year)],
        risk_free=env._load_risk_free().squeeze().loc[str(year)],
    )
    renderer.plotly_report()
    renderer.tearsheet()
    
    levels.append(renderer.level.to_frame().pct_change())
    
    
    
    actions = pd.DataFrame(episode.actions, index=episode.time, columns=env.action_space.contracts[1:])
    states = pd.DataFrame(episode.states, index=episode.time)
    target_russell = actions[actions.columns[0]]
    mapping = states[0].to_frame('GAIA Predictor').join(target_russell)
    mappings = mappings.append(mapping)
    
    
    
    this_mapping = dict()
    env.reset()
    for predictor in np.linspace(-0.01, 0.01, 500):
        from trading_gym.registry.gaia.v7.env import EventGAIAPredictor
        from scipy.special import softmax
        event = EventGAIAPredictor(time=datetime.now(), equities=predictor)
        env.add_event(event)
        state = env.state.get_values()

        _, _, info = policy.compute_single_action(state, [])
        logits = info['behaviour_logits']
        action = softmax(logits)
        target_weight_equity = action[0]
        this_mapping[predictor] = target_weight_equity
    mapping_functions[year] = this_mapping

In [ ]:

mappings.sort_index().iplot(
#     secondary_y='Russell 1000'  ---    This was here before
    secondary_y=str(mappings.columns[-1]),
    vline=[str(year) for year in range(2008, 2019)]
)

In [ ]:
pd.DataFrame(mapping_functions).iplot(title='Mapping functions trained with deep reinforcement learning', yTitle='Target weight for Russell 1000', xTitle='Standardized GAIA predictor')

In [ ]:
mappings.set_index('GAIA Predictor').iplot(kind='scatter', mode='markers', size=3, yTitle='Target weight for Russell 1000', xTitle='Standardized GAIA predictor')

(pid=70635) 2019-07-05 10:10:04,673	INFO policy_evaluator.py:311 -- Creating policy evaluation worker 3 on CPU (please ignore any CUDA init errors)
(pid=70637) 2019-07-05 10:10:04,687	INFO policy_evaluator.py:311 -- Creating policy evaluation worker 4 on CPU (please ignore any CUDA init errors)
(pid=70637) 2019-07-05 10:10:04.714207: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=70635) 2019-07-05 10:10:04.746383: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=108336) 2019-07-05 10:10:06,945	INFO policy_evaluator.py:311 -- Creating policy evaluation worker 5 on CPU (please ignore any CUDA init errors)
(pid=108339) 2019-07-05 10:10:06,920	INFO policy_evaluator.py:311 -- Creating policy evaluation worker 6 on CPU (please ignore any CUDA init errors)
(pid=108339) 2019-07-05 10:10:06.9